# Final Project - SVM Soft Margin Extension - Numpy

In [2]:
import numpy as np
import csv
import math
from numpy import genfromtxt
from sklearn.datasets import load_digits
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from cvxopt import matrix, solvers
import matplotlib.pyplot as plt
%matplotlib inline

## 1. MNIST

### Homework implementation

In [3]:
digits=load_digits()
X = digits.data
y = digits.target

# Scale training features
X_scale = StandardScaler()
X = X_scale.fit_transform(digits.data)

In [4]:
# Assign X and y the subset of data that describe the numbers 8 and 9

new_X = []
new_y = []
for i in range(len(X)):
    if y[i] == 8:
        new_X.append(X[i])
        new_y.append(y[i])
    elif y[i] == 9:
        new_X.append(X[i])
        new_y.append(y[i])
new_X = np.array(new_X)
new_y = np.array(new_y)

X = new_X
y = new_y

In [5]:
# Train-test split

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.6,random_state=42)


In [6]:
print(X_train.shape)
print(y_train.shape)
print(X_test.shape)
print(y_test.shape)

(141, 64)
(141,)
(213, 64)
(213,)


In [7]:
y_train.shape

(141,)

In [8]:
def kernel_svm(X, y): 

    m,n = X.shape
    y = y.reshape(-1,1)
    X_y = X*y
    H = np.dot(X_y, X_y.T)
    
    P = matrix(H)
    q = matrix(-np.ones((m, 1)))
    G = matrix(-np.eye(m))
    h = matrix(np.zeros(m))
    A = matrix(y.reshape(1,-1))
    A = matrix(A, (1, m), 'd')
    b = matrix(np.zeros(1))
    
    sol = solvers.qp(P,q,G,h,A,b) 
    
    alphas = np.array(sol['x'])[:,0]
    
    return alphas

# fit svm dual classifier
alphas = kernel_svm(X_train, y_train)

     pcost       dcost       gap    pres   dres
 0: -3.8262e-02 -4.0188e-02  2e+02  1e+01  1e+00
 1: -3.0905e-04 -1.1130e-04  2e+00  1e-01  1e-02
 2: -1.8827e-05 -6.7504e-05  3e-02  2e-03  2e-04
 3: -6.8873e-06 -1.1057e-05  1e-03  8e-05  7e-06
 4: -1.2441e-06 -1.0286e-07  3e-05  3e-06  2e-07
 5: -1.3608e-08 -1.0581e-11  4e-07  3e-08  2e-09
 6: -1.3618e-10 -1.0581e-15  4e-09  3e-10  2e-11
Optimal solution found.


In [9]:
def kernel_svm(X, y): 

    m,n = X.shape
    y = y.reshape(-1,1)*1.
    X_y = X*y
    H = np.dot(X_y, X_y.T)
    
    P = matrix(H)
    q = matrix(-np.ones((m, 1)))
    G = matrix(-np.eye(m))
    h = matrix(np.zeros(m))
    A = matrix(y.reshape(1,-1))
    A = matrix(A, (1, m), 'd')
    b = matrix(np.zeros(1))
    
    sol = solvers.qp(P,q,G,h,A,b) 
    
    alphas = np.array(sol['x'])[:,0]
    
    return alphas

# fit svm dual classifier
alphas = kernel_svm(X_train, y_train)

print(alphas)

     pcost       dcost       gap    pres   dres
 0: -3.8262e-02 -4.0188e-02  2e+02  1e+01  1e+00
 1: -3.0905e-04 -1.1130e-04  2e+00  1e-01  1e-02
 2: -1.8827e-05 -6.7504e-05  3e-02  2e-03  2e-04
 3: -6.8873e-06 -1.1057e-05  1e-03  8e-05  7e-06
 4: -1.2441e-06 -1.0286e-07  3e-05  3e-06  2e-07
 5: -1.3608e-08 -1.0581e-11  4e-07  3e-08  2e-09
 6: -1.3618e-10 -1.0581e-15  4e-09  3e-10  2e-11
Optimal solution found.
[ 1.10008300e-11  2.09343171e-11 -1.52888491e-11 -1.67481751e-11
 -1.66363190e-11 -1.37071990e-11 -1.49454228e-11 -4.22863306e-12
 -1.65478139e-11  3.43433702e-11  2.60853523e-11  2.11618856e-11
 -1.54137825e-11 -1.85374511e-11 -1.44943188e-11  2.11612318e-11
  2.58213534e-11 -1.36190907e-11 -1.72111703e-11 -1.74237986e-11
  1.34051841e-11  8.87109839e-12 -1.49446968e-11 -1.58828641e-11
 -1.29457904e-11  1.39840867e-11  2.43037776e-11  2.68312148e-11
  1.28495891e-11 -9.40838869e-12 -1.56115124e-11  2.20192537e-11
  2.57802252e-11 -1.64524268e-11  1.20248647e-11  8.71851732e-12


In [10]:
def compute_classification_boundary (X, y, alpha):
    #cond = (alphas > 1e-3).reshape(-1)
    cond = [i for i in range(len(alphas)) if alphas[i] > 1e-12]
    w = np.dot(X.T, alpha*y).reshape(-1,1)
    w0 = y[cond] - np.dot(X[cond], w)
    w0 = np.mean(w0)
    return w, w0



w, w0 = compute_classification_boundary(X_train, y_train, alphas)

In [11]:
# Determine which training examples are support vectors
support_vectors = []

for i in range(len(alphas)):
    if alphas[i] > 1e-12:
        support_vectors.append([X_train[i], y_train[i], i])

In [12]:
def K(xi, xj):
    return np.dot(xi,xj)

alpha_indices = [support_vectors[i][2] for i in range(len(support_vectors))]
print(alpha_indices)

[0, 1, 9, 10, 11, 15, 16, 20, 21, 25, 26, 27, 28, 31, 32, 34, 35, 40, 45, 46, 50, 53, 54, 55, 57, 58, 60, 61, 63, 64, 65, 67, 69, 73, 75, 76, 79, 83, 86, 89, 91, 95, 96, 103, 104, 107, 108, 110, 114, 117, 118, 120, 121, 125, 126, 127, 128, 129, 130, 131, 133, 134, 135, 136, 137, 138, 139, 140]


In [13]:
def f_dual(x):
    summation = 0
    for i in range(len(support_vectors)):
        summation += alphas[alpha_indices[i]]*y_train[alpha_indices[i]]*K(X_train[alpha_indices[i]],x)
    if (summation >= 0):
        return 8
    else:
        return 9

In [14]:
# Test SVM dual classifier on X_test

def predict(X):
    predictions = []
    for i in range(len(X_test)):
        predictions.append(f_dual(X_test[i]))
    return predictions

y_pred = predict(X_test)

In [15]:
# Print accuracy

print('Prediction accuracy is {}%'.format(accuracy_score(y_test, y_pred) * 100))

Prediction accuracy is 79.81220657276995%


### Extended implementation using Numpy

In [16]:
def kernel_soft_margin_svm(X, y, C): 

    m,n = X.shape
    y = y.reshape(-1,1)
    X_y = X*y
    H = np.dot(X_y, X_y.T)
    
    P = matrix(H)
    q = matrix(-np.ones((m, 1)))
    
    # Changed G and h
    G = matrix(np.vstack((np.diag(np.ones(m))*-1, np.identity(m))))
    h = matrix(np.hstack((np.zeros(m), np.ones(m)*C)))
    
    A = matrix(y.reshape(1,-1))
    A = matrix(A, (1, m), 'd')
    b = matrix(np.zeros(1))
    
    sol = solvers.qp(P,q,G,h,A,b) 
    
    alphas = np.array(sol['x'])[:,0]
    
    return alphas

# fit svm dual classifier
alphas = kernel_soft_margin_svm(X_train, y_train, 0.1)

     pcost       dcost       gap    pres   dres
 0: -2.1056e-02 -1.4523e+01  3e+02  1e+01  3e-14
 1: -3.6794e-03 -1.2104e+01  2e+01  2e-01  3e-14
 2: -1.3539e-05 -4.4261e-01  5e-01  4e-03  5e-15
 3: -6.7445e-06 -1.3705e-02  1e-02  1e-04  6e-16
 4: -1.4392e-06 -5.3602e-04  6e-04  4e-06  5e-16
 5: -1.6065e-08 -5.4504e-06  6e-06  4e-08  8e-16
 6: -1.6080e-10 -5.4504e-08  6e-08  4e-10  8e-16
Optimal solution found.


In [17]:
print(alphas)

[ 1.36585483e-11  2.27506384e-11 -1.92366639e-11 -1.98397047e-11
 -2.00293069e-11 -1.61368576e-11 -1.81284643e-11  2.00850294e-12
 -2.03122376e-11  4.36445838e-11  2.90944943e-11  2.33602023e-11
 -1.95539561e-11 -2.12733573e-11 -1.69055886e-11  2.44005157e-11
  2.84644718e-11 -1.60393180e-11 -2.08773643e-11 -2.05332723e-11
  1.58030199e-11  1.03761550e-11 -1.75146329e-11 -1.81462117e-11
 -1.64446170e-11  1.71012765e-11  2.81641735e-11  3.21843997e-11
  1.55842114e-11 -1.12318390e-11 -1.92758127e-11  2.37963252e-11
  3.03680369e-11 -1.87172034e-11  1.57760173e-11  1.12448786e-11
 -2.04423767e-11 -1.73569720e-11 -2.05143741e-11 -1.88305274e-11
  2.32417801e-11 -1.36254299e-11 -1.68648253e-11 -2.06368812e-11
 -1.46642733e-11  1.76430957e-11  2.20579040e-11 -1.91555891e-11
 -1.67150386e-11 -1.85446111e-11  1.14276482e-11 -2.02973481e-11
 -1.04092602e-11  2.27294890e-11  3.06090172e-11  7.51381618e-12
 -2.00792956e-11  1.47062924e-11  1.82162155e-11 -1.79397805e-11
  3.02387562e-11  2.31590

In [18]:
def compute_classification_boundary (X, y, alpha):
    #cond = (alphas > 1e-3).reshape(-1)
    cond = [i for i in range(len(alphas)) if alphas[i] > 1e-12]
    w = np.dot(X.T, alpha*y).reshape(-1,1)
    w0 = y[cond] - np.dot(X[cond], w)
    w0 = np.mean(w0)
    return w, w0



w, w0 = compute_classification_boundary(X_train, y_train, alphas)

In [19]:
# Determine which training examples are support vectors
support_vectors = []

for i in range(len(alphas)):
    if alphas[i] > 1e-12:
        support_vectors.append([X_train[i], y_train[i], i])

# print("The following are support vectors: ")
# for i in range(len(support_vectors)):
#     print(support_vectors[i][0])

In [20]:
def K(xi, xj):
    return np.dot(xi,xj)

alpha_indices = [support_vectors[i][2] for i in range(len(support_vectors))]
print(alpha_indices)

[0, 1, 7, 9, 10, 11, 15, 16, 20, 21, 25, 26, 27, 28, 31, 32, 34, 35, 40, 45, 46, 50, 53, 54, 55, 57, 58, 60, 61, 63, 64, 65, 67, 69, 73, 75, 76, 79, 83, 86, 89, 91, 95, 96, 103, 104, 107, 108, 110, 114, 117, 118, 120, 121, 125, 126, 127, 128, 129, 130, 131, 133, 134, 135, 136, 137, 138, 139, 140]


In [21]:
def f_dual(x):
    summation = 0
    for i in range(len(support_vectors)):
        summation += alphas[alpha_indices[i]]*y_train[alpha_indices[i]]*K(X_train[alpha_indices[i]],x)
    if (summation >= 0):
        return 8
    else:
        return 9

In [22]:
# Test SVM dual classifier on X_test

def predict(X):
    predictions = []
    for i in range(len(X_test)):
        predictions.append(f_dual(X_test[i]))
    return predictions

y_pred = predict(X_test)

In [23]:
# Print accuracy

print('Prediction accuracy is {}%'.format(accuracy_score(y_test, y_pred) * 100))

Prediction accuracy is 80.28169014084507%


## 2. Fashion-MNIST

### Homework implementation

In [24]:
from keras.datasets import fashion_mnist
((trainX, trainY), (testX, testY)) = fashion_mnist.load_data()

Using TensorFlow backend.


In [25]:
X_train = trainX
y_train = trainY
X_test = testX
y_test = testY

In [26]:
# Assign X_train and y_train the subset of data that describe the labels 0 and 2 (T-shirts and pullovers, respectively)

new_X_train = []
new_y_train = []
for i in range(len(X_train)):
    if y_train[i] == 0:
        new_X_train.append(X_train[i])
        new_y_train.append(y_train[i])
    elif y_train[i] == 2:
        new_X_train.append(X_train[i])
        new_y_train.append(y_train[i])
new_X_train = np.array(new_X_train)
new_y_train = np.array(new_y_train)

X_train = new_X_train
y_train = new_y_train

In [27]:
# Assign X_test and y_test the subset of data that describe the labels 0 and 2 (T-shirts and pullovers, respectively)

new_X_test = []
new_y_test = []
for i in range(len(X_test)):
    if y_test[i] == 0:
        new_X_test.append(X_test[i])
        new_y_test.append(y_test[i])
    elif y_test[i] == 2:
        new_X_test.append(X_test[i])
        new_y_test.append(y_test[i])
new_X_test = np.array(new_X_test)
new_y_test = np.array(new_y_test)

X_test = new_X_test
y_test = new_y_test

In [28]:
X_train = np.array([X_train[i].flatten() for i in range(len(X_train))])
X_test = np.array([X_test[i].flatten() for i in range(len(X_test))])

print(X_train.shape)
print(y_train.shape)
print(X_test.shape)
print(y_test.shape)

(12000, 784)
(12000,)
(2000, 784)
(2000,)


In [29]:
# Downsample the data

# Add y_train back as an additional column to X_train
y_train = y_train.reshape((-1,1))
X_train = np.append(X_train, y_train, axis=1)

# Add y_test back as an additional column to X_test
y_test = y_test.reshape((-1,1))
X_test = np.append(X_test, y_test, axis=1)

# Shuffle the data
np.random.shuffle(X_train)
np.random.shuffle(X_test)

# Slice out only the first 141 from X_train and 213 from X_test
X_train = X_train[0:141]
X_test = X_test[0:213]

# Remove the last columns of X_train and X_test and place them back into y_train and y_test
y_train = X_train[:,-1]
y_test = X_test[:,-1]
X_train = X_train[:,0:X_train.shape[1]-1]
X_test = X_test[:,0:X_test.shape[1]-1]

print(X_train.shape)
print(y_train.shape)
print(X_test.shape)
print(y_test.shape)

(141, 784)
(141,)
(213, 784)
(213,)


In [30]:
# Scale the dataset

X_scale = StandardScaler()
X_train = X_scale.fit_transform(X_train) 
X_test = X_scale.fit_transform(X_test) 

In [31]:
def kernel_svm(X, y): 

    m,n = X.shape
    y = y.reshape(-1,1)*1.
    X_y = y*X
    H = np.dot(X_y, X_y.T)*1.

    P = matrix(H)
    q = matrix(-np.ones((m, 1)))
    G = matrix(-np.eye(m))
    h = matrix(np.zeros(m))
    A = matrix(y.reshape(1,-1))
    A = matrix(A, (1, m), 'd')
    b = matrix(np.zeros(1))

    
    sol = solvers.qp(P,q,G,h,A,b) 
    
    alphas = np.array(sol['x'])[:,0]
    
    return alphas

# fit svm dual classifier
alphas = kernel_svm(X_train, y_train)

print(alphas)

     pcost       dcost       gap    pres   dres
 0: -6.9000e+01 -1.3800e+02  3e+02  1e+01  2e+00
 1: -3.0922e+02 -3.1231e+02  8e+01  6e+00  1e+00
 2: -3.4741e+04 -3.4744e+04  8e+01  6e+00  1e+00
 3: -3.4434e+08 -3.4434e+08  4e+02  6e+00  1e+00
 4: -3.4090e+14 -3.4090e+14  3e+06  6e+00  1e+00
 5: -3.3749e+22 -3.3749e+22  3e+12  4e+00  1e+00
 6: -3.3412e+32 -3.3412e+32  3e+20  9e+15  1e+00
 7: -3.3075e+44 -3.3075e+44  3e+30  5e+27  1e+00
 8: -3.3278e+58 -3.3278e+58  3e+42  1e+42  1e+00
 9: -6.4522e+74 -6.4522e+74  6e+56  1e+58  1e+00
10: -6.3877e+110 -2.0609e+112  2e+112  9e+93  2e+01
11: -6.7860e+110 -8.9075e+110  2e+110  2e+94  9e-01
12: -6.7860e+110 -8.9075e+110  2e+110  2e+94  9e-01
13: -6.7860e+110 -8.9075e+110  2e+110  2e+94  1e+00
14: -6.7860e+110 -8.9075e+110  2e+110  2e+94  3e+00
15: -6.7860e+110 -8.9075e+110  2e+110  2e+94  6e+00
16: -6.7860e+110 -8.9075e+110  2e+110  2e+94  2e+01
17: -6.7860e+110 -8.9075e+110  2e+110  2e+94  6e+01
18: -6.7860e+110 -8.9075e+110  2e+110  2e+94  

In [32]:
def compute_classification_boundary (X, y, alpha):
    #cond = (alphas > 1e-3).reshape(-1)
    cond = [i for i in range(len(alphas)) if alphas[i] < 1e-7]
    w = np.dot(X.T, alpha*y).reshape(-1,1)
    w0 = y[cond] - np.dot(X[cond], w)
    w0 = np.mean(w0)
    return w, w0



w, w0 = compute_classification_boundary(X_train, y_train, alphas)

In [33]:
# Determine which training examples are support vectors
support_vectors = []

for i in range(len(alphas)):
    if alphas[i] < 1e-7:
        support_vectors.append([X_train[i], y_train[i], i])

In [34]:
def K(xi, xj):
    return np.dot(xi,xj)

alpha_indices = [support_vectors[i][2] for i in range(len(support_vectors))]
print(alpha_indices)

[0, 5, 7, 8, 9, 11, 14, 15, 17, 18, 20, 22, 23, 26, 27, 28, 29, 32, 33, 36, 40, 50, 58, 59, 60, 65, 68, 69, 72, 73, 74, 75, 76, 82, 84, 85, 86, 92, 94, 97, 98, 100, 103, 107, 109, 110, 113, 117, 119, 120, 121, 124, 127, 128, 130, 132, 134, 135, 136, 137, 138, 139]


In [35]:
def f_dual(x):
    summation = 0
    for i in range(len(support_vectors)):
        summation += alphas[alpha_indices[i]]*y_train[alpha_indices[i]]*K(X_train[alpha_indices[i]],x)
    if (summation >= 0):
        return 0
    else:
        return 2

In [36]:
# Test SVM dual classifier on X_test

def predict(X):
    predictions = []
    for i in range(len(X_test)):
        predictions.append(f_dual(X_test[i]))
    return predictions

y_pred = predict(X_test)

In [37]:
# Print accuracy

print('Prediction accuracy is {}%'.format(accuracy_score(y_test, y_pred) * 100))

Prediction accuracy is 94.83568075117371%


### Extended implementation using Numpy

In [38]:
def kernel_soft_margin_svm(X, y, C): 

    m,n = X.shape
    y = y.reshape(-1,1)*1.
    X_y = X*y
    H = np.dot(X_y, X_y.T)*1.
    
    P = matrix(H)
    q = matrix(-np.ones((m, 1)))
    
    # Changed G and h
    G = matrix(np.vstack((np.diag(np.ones(m))*-1, np.identity(m))))
    h = matrix(np.hstack((np.zeros(m), np.ones(m)*C)))
    
    A = matrix(y.reshape(1,-1))
    A = matrix(A, (1, m), 'd')
    b = matrix(np.zeros(1))
    
    sol = solvers.qp(P,q,G,h,A,b) 
    
    alphas = np.array(sol['x'])[:,0]
    
    return alphas

# fit svm dual classifier
alphas = kernel_soft_margin_svm(X_train, y_train, 0.1)

     pcost       dcost       gap    pres   dres
 0: -3.7950e+01 -2.4240e+01  6e+02  2e+01  2e-16
 1: -5.8654e+00 -2.2413e+01  2e+01  2e-01  5e-16
 2: -6.3600e+00 -7.4989e+00  1e+00  9e-03  4e-16
 3: -6.8946e+00 -6.9069e+00  1e-02  9e-05  3e-16
 4: -6.8999e+00 -6.9001e+00  1e-04  9e-07  2e-16
 5: -6.9000e+00 -6.9000e+00  1e-06  9e-09  3e-16
Optimal solution found.


In [39]:
print(alphas)

[ 2.49642095e-24  9.99999922e-02  9.99999922e-02  9.99999922e-02
  9.99999922e-02  4.54330640e-24  9.99999922e-02  1.94066816e-24
  2.10612603e-24 -1.18189551e-23  9.99999922e-02  2.62474435e-24
  9.99999922e-02  9.99999922e-02 -4.99045415e-24 -1.86933099e-24
  9.99999922e-02  1.05629820e-23 -6.78707232e-24  9.99999922e-02
 -3.37181898e-24  9.99999922e-02 -4.80846074e-24  3.67788149e-24
  9.99999922e-02  9.99999922e-02  6.20992933e-24 -5.94485071e-24
  3.23737956e-24  1.84407469e-24  9.99999922e-02  9.99999922e-02
  1.11701552e-23  1.75627956e-23  9.99999922e-02  9.99999922e-02
  2.53331950e-24  9.99999922e-02  9.99999922e-02  9.99999922e-02
 -6.42898062e-24  9.99999922e-02  7.40177573e-24  9.99999922e-02
  9.99999922e-02  9.99999922e-02 -7.93460148e-24  9.99999922e-02
  9.99999922e-02  1.26553724e-23 -3.81814179e-24  9.99999922e-02
  3.07022333e-24  9.99999922e-02  9.99999922e-02  9.99999922e-02
  9.99999922e-02  9.99999922e-02 -5.41194888e-24 -5.90645082e-24
 -2.06147387e-24 -2.76313

In [40]:
def compute_classification_boundary (X, y, alpha):
    #cond = (alphas > 1e-3).reshape(-1)
    cond = [i for i in range(len(alphas)) if alphas[i] < 1e-2]
    w = np.dot(X.T, alpha*y).reshape(-1,1)
    w0 = y[cond] - np.dot(X[cond], w)
    w0 = np.mean(w0)
    return w, w0



w, w0 = compute_classification_boundary(X_train, y_train, alphas)

In [41]:
# Determine which training examples are support vectors
support_vectors = []

for i in range(len(alphas)):
    if alphas[i] < 1e-2:
        support_vectors.append([X_train[i], y_train[i], i])

# print("The following are support vectors: ")
# for i in range(len(support_vectors)):
#     print(support_vectors[i][0])

In [42]:
def K(xi, xj):
    return np.dot(xi,xj)

alpha_indices = [support_vectors[i][2] for i in range(len(support_vectors))]
print(alpha_indices)

[0, 5, 7, 8, 9, 11, 14, 15, 17, 18, 20, 22, 23, 26, 27, 28, 29, 32, 33, 36, 40, 42, 46, 49, 50, 52, 58, 59, 60, 61, 65, 68, 69, 72, 73, 74, 75, 76, 82, 84, 85, 86, 89, 90, 92, 94, 97, 98, 99, 100, 102, 103, 107, 109, 110, 113, 117, 119, 120, 121, 124, 127, 128, 129, 130, 132, 134, 135, 136, 137, 138, 139]


In [43]:
def f_dual(x):
    summation = 0
    for i in range(len(support_vectors)):
        summation += alphas[alpha_indices[i]]*y_train[alpha_indices[i]]*K(X_train[alpha_indices[i]],x)
    if (summation >= 0):
        return 0
    else:
        return 2

In [44]:
# Test SVM dual classifier on X_test

def predict(X):
    predictions = []
    for i in range(len(X_test)):
        predictions.append(f_dual(X_test[i]))
    return predictions

y_pred = predict(X_test)

In [45]:
# Print accuracy

print('Prediction accuracy is {}%'.format(accuracy_score(y_test, y_pred) * 100))

Prediction accuracy is 47.88732394366197%
